In [1]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.notebook.scroll_to_top();
                jupyter.actions.call('jupyter-notebook:save-notebook');                
                
            });
        });
        
        $( document ).ready(function(){
        code_shown=false;
        $('div.input').hide()});
    
    
</script>

Note: Above this cell is a hidden cell that hides and runs all code in the file. This is intended for those who do not want to see or interact with the code. It can be seen by converting the cell to markdown(see toolbar above) and then back to code

# Density and Demographic Merge

The following code merges the json files downloaded in the Density Exploration and Conversion and the Demographic Exploration and Conversion files. 

- The output is a table with latitude and lognitude (to include web formats), population and demographic distribution for the desired area. 
 
- The desired area is set via Density Exploration and Conversion file. 
- Both outputs from the Density and Demographic files need to have the same accuracy (e.g. 0.11 meter, 1 kilometer etc) 

# 0. Import Dependencies

In [2]:
from toggle_code import toggle_code as hide_code
from toggle_code import run_code as run_code
import pandas as pd
import json
import glob
import ipywidgets as widgets
from ipywidgets import interact, Layout
import re
import numpy as np
import random
import os

print ("Dependencies imported")

Dependencies imported


# 1. Select the population density file you want to merge with demographics files.



In [3]:
hide_code()
run_code()

#get files
files = ["None Selected"]
filepath= r"./data/*"
for pop_file in glob.glob(filepath):
    if ".csv" in pop_file or ".json" in pop_file: 
        files.append(pop_file)

print("Select the output from the Density Exploration and Conversion file:")
pop_file = widgets.Select(options=files, value=files[0], 
                                  description="File: ",
                                  disabled = False,
                                  layout=Layout(width="50%", height="260px"))

def update(file):
    return file

pop_file_select = interact(update, file=pop_file)
      

Select the output from the Density Exploration and Conversion file:


interactive(children=(Select(description='File: ', layout=Layout(height='260px', width='50%'), options=('None …

# 2. Create base table from density file

In [4]:
hide_code()
run_code()

main = pop_file.value
main_key = ""
ref = {}
syn_pop = "Waiting for Inputs"

if main == 'None Selected':
    pass

else:
    print("Getting base table.")
    for file in files: 
        if file == 'None Selected' or "json" not in file:
            pass
        else: 
            #get key for dict
            key = re.split(r'\\', file)
            key = key[1].split('.')
            ref[key[0]]= pd.read_json(file)
            if key[0] in main: 
                main_key = key[0]

    syn_pop = ref.pop(main_key)
syn_pop

'Waiting for Inputs'

# 3. Merge demographic files into one dataframe

In [5]:
hide_code()
run_code()

if main == 'None Selected':
    pass

else:
    for key,value in ref.items(): 
            print("Merging "+ key)
            del value["Percent"]
            syn_pop = pd.merge(syn_pop, value, how ="left", on=["latitude", "longitude"])


        #del syn_pop["Population_density"]

syn_pop

'Waiting for Inputs'

# 4. Compare population density to demographic totals 

In [6]:
hide_code()
run_code()


if main == 'None Selected':
    print ("Waiting for inputs")

else: 
    syn_pop = syn_pop.replace(np.nan,0)
    syn_pop["dem_total"] = syn_pop[syn_pop.columns[6:]].sum(axis=1)
    syn_pop

    total = syn_pop["Population"].sum()
    dem_total = syn_pop["dem_total"].sum()
    print ("Your population density total is {}".format(total))
    print("Your demographic total is {}".format(dem_total))

    def fill_dems(row, diff):
        while diff>0:
            choice = random.choice(list(syn_pop.columns[6:-2]))
            row[choice] += 1
            diff -= 1

    if total == dem_total:
        print("Density and Demographic totals are equal no further action required.")
    else: 
        if total > dem_total: #changes 
            print("The population total is more thant the demogrpahic total, this will take awhile to compute...")
            #TODO Come up with better way
            syn_pop['fill'] = np.where((syn_pop['dem_total'] < syn_pop['Population']),
                                       syn_pop['Population']-syn_pop["dem_total"], 0) 
            for idx, row in syn_pop.iterrows():
                if row["fill"] >0:
                    fill_dems(row, row["fill"])   
        else: 
            print("Your demographic population is more than your population density.")
            print("Your population density now reflects the demographic data.")
            syn_pop["Population"] = syn_pop["dem_total"]
            del syn_pop["dem_total"]



Waiting for inputs


# 5. How do you want to save your file?

In [7]:
hide_code()
run_code()


if main == 'None Selected':
    print("Waiting for Inputs")
else:
    print("Do you want to drop 0 values to make a smaller file?")
    drop =["Yes", "No"]
    drop_0 = widgets.Dropdown(options=drop,
                              value= drop[0],                        
                              description="Drop: ",
                              disabled = False)

    def update1(Drop):
        return Drop


    drop_select = interact(update1, Drop= drop_0)

    print("Do you want to save as a csv or json?")

    file_type = ["csv", "json"]

    file_select = widgets.Dropdown(options=file_type,
                                   value = file_type[0],
                                   description="File",disabled = False)

    def update2(File_type):
        return File_type



    file_selected = interact(update2, File_type= file_select)


Waiting for Inputs


# 6. Save your file

In [8]:
hide_code()
run_code()


if main == 'None Selected':
    print("Waiting for inputs")
else:
    if drop_0.value == "Yes":
        syn_pop = syn_pop[syn_pop["Population"]!=0]

    filepath = os.path.join(r".\data", "synthetic_population")

    if file_select.value == "csv":
        filepath =filepath+".csv"
        syn_pop.to_csv(filepath)
        print(filepath + " is saved.")
    else: 
        filepath =filepath+".json"
        syn_pop.to_json(filepath)
        print(filepath + " is saved")

Waiting for inputs
